In [1]:
import pandas as pd

In [2]:
with open('yelp_academic_dataset.json', 'rb') as f:
    data = f.readlines()
# remove "\n"s
data = map(lambda x: x.rstrip(), data)
# convert to json string
data_json_str = "[" + ','.join(data) + "]"
# load as pandas df
rawdata = pd.read_json(data_json_str)

In [3]:
rawdata.columns

Index([u'average_stars',   u'business_id',    u'categories',          u'city',
                u'date',  u'full_address',      u'latitude',     u'longitude',
                u'name', u'neighborhoods',          u'open',     u'photo_url',
        u'review_count',     u'review_id',       u'schools',         u'stars',
               u'state',          u'text',          u'type',           u'url',
             u'user_id',         u'votes'],
      dtype='object')

In [4]:
rawdata.shape

(474434, 22)

In [5]:
# make reviews dataframe by removing rows with NaN in text column
reviews = rawdata[pd.notnull(rawdata['text'])]
reviews = reviews.reset_index(drop=True)
reviews = reviews.dropna(axis=1,how='all')

In [6]:
# make business dataframe by removing rows with NaN in categories column
business = rawdata[pd.notnull(rawdata['categories'])]
business = business.reset_index(drop=True)
business = business.dropna(axis=1,how='all')

In [7]:
# select businesses that have 'Restaurants' in their category and have >100 reviews
restaurant_index = []
for i in range(0,len(business)):
    if 'Restaurants' in business.categories[i] and business.review_count[i] > 100 : 
              restaurant_index.append(i)
business_restaurant = business.iloc[restaurant_index] 

In [8]:
business_restaurant.head(3)

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,photo_url,review_count,schools,stars,state,type,url
54,81IjU5L-t-QQwsE38C63hQ,"[Burgers, Gastropubs, Restaurants]",Claremont,"128 N Oberlin Ave\nClaremont, CA 91711",34.094884,-117.720685,The Back Abbey,[],1,http://s3-media4.ak.yelpcdn.com/bphoto/19sYSVQ...,756,[Harvey Mudd College],4.0,CA,business,http://www.yelp.com/biz/the-back-abbey-claremont
83,8lLs3dsSN-Am2_EtNfbXqA,"[Breakfast & Brunch, American (New), Restaurants]",New York,"2893 Broadway\nMorningside Heights\nNew York, ...",40.805798,-73.965675,Community Food & Juice,[Morningside Heights],1,http://s3-media2.ak.yelpcdn.com/bphoto/IqKj8kM...,346,[Columbia University],3.5,NY,business,http://www.yelp.com/biz/community-food-and-jui...
122,7sBIv30fhszkBsv0kkSBCg,"[Tex-Mex, Restaurants]",Seattle,2614 NE 46th St\nUniversity Village\nUniversit...,47.661733,-122.299277,Sonrisa,[University District],1,http://s3-media1.ak.yelpcdn.com/bphoto/qvFsHdP...,120,[University of Washington],3.0,WA,business,http://www.yelp.com/biz/sonrisa-seattle


In [9]:
# total number of restaurants, mean number of reviews per restaurant, total number of reviews
len(business_restaurant),business_restaurant['review_count'].mean(), business_restaurant['review_count'].sum()

(616, 222.58928571428572, 137115.0)

In [10]:
# create df of restaurant business ids
busid = pd.DataFrame(business_restaurant['business_id'])

In [11]:
busid.head(3)

,business_id
54,81IjU5L-t-QQwsE38C63hQ
83,8lLs3dsSN-Am2_EtNfbXqA
122,7sBIv30fhszkBsv0kkSBCg


In [12]:
# total number of reviews in each segment
len(reviews)

330071

In [13]:
# extract restaurant reviews only
review_restaurant = pd.merge(busid,reviews,how='inner',on='business_id')

In [14]:
# number of restaurant reviews in each segment
len(review_restaurant)

127753

In [15]:
# number of restaurants
# sanity check: = len(business_restaurant)
len(review_restaurant.business_id.unique())

616

In [16]:
# sort reviews by restaurant
test = review_restaurant.groupby('business_id').count()
test = test.sort('date',ascending=0)

In [17]:
test.tail()

,date,review_id,stars,text,type,user_id,votes
business_id,,,,,,,
2WXwWfhpfia7_UflDYnUyQ,92,92,92,92,92,92,92
Ea-ZuT5DKD188BpRMTDyGA,92,92,92,92,92,92,92
XO6V1_V6PXOpml8xoq6-YQ,91,91,91,91,91,91,91
HdGDvsK8jEdstSS5g21FDg,91,91,91,91,91,91,91
TKDgFp9C7kf_pSHrQHZW_Q,88,88,88,88,88,88,88


In [18]:
# review segment 1
# number of restaurants with >100 reviews, total number of reviews, mean number of reviews per restaurant
test['date'][test['date']>100].count(), test['date'][test['date']>100].sum(), test['date'][test['date']>100].mean()

(563, 122657, 217.86323268206039)

In [19]:
# get list of business ids with > 100 reviews in reviews json file
busid_100 = pd.DataFrame(test[test['date']>100].index.values, columns=['business_id'])

In [20]:
# extract the business and reviews info of all businesses in busid_100
business_final = pd.merge(busid_100,business_restaurant,how='inner',on='business_id')
reviews_final = pd.merge(busid_100,review_restaurant,how='inner',on='business_id')

In [21]:
len(business_final), len(reviews_final)

(563, 122657)

In [24]:
business_final.head(3)

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,photo_url,review_count,schools,stars,state,type,url
0,qHmamQPCAKkia9X0uryA8g,"[Restaurants, Hot Dogs]",Berkeley,"2534 Durant Ave\nTelegraph Ave\nBerkeley, CA 9...",37.867894,-122.257433,Top Dog,"[Telegraph Ave, UC Campus Area]",1,http://s3-media4.ak.yelpcdn.com/bphoto/nFfca2I...,1270,[University of California at Berkeley],4.5,CA,business,http://www.yelp.com/biz/top-dog-berkeley
1,JrGSfjRqAtIZjjwC3FFQ4w,"[American (Traditional), Barbeque, Restaurants]",San Luis Obispo,"1001 Higuera St\nSte A\nSan Luis Obispo, CA 93401",35.281110,-120.660760,Firestone Grill,[],1,http://s3-media4.ak.yelpcdn.com/bphoto/p3Gp2uN...,1153,[California Polytechnic State University],4.5,CA,business,http://www.yelp.com/biz/firestone-grill-san-lu...
2,VFslQjSgrw4Mu5_Q1xk1KQ,"[Steakhouses, Brazilian, Restaurants]",Palo Alto,"529 Alma St\nPalo Alto, CA 94301",37.442975,-122.163048,Pampas,[],1,http://s3-media4.ak.yelpcdn.com/bphoto/Wn0OeAX...,1146,[Stanford University],4.0,CA,business,http://www.yelp.com/biz/pampas-palo-alto


In [25]:
reviews_final.head(3)

,business_id,date,review_id,stars,text,type,user_id,votes
0,qHmamQPCAKkia9X0uryA8g,2006-09-23,M8G9Rs21i4euIo3T5gyGOg,4,Are you drunk? Is it around 3am? Are you in do...,review,Xsp1amevfceAqAMjKhZkgA,"{u'funny': 0, u'useful': 1, u'cool': 0}"
1,qHmamQPCAKkia9X0uryA8g,2011-04-06,o9HYfNDSACBPRykq8t21PQ,3,OH NO HE DIDN'T.\n\nTHE GUY MAKING THE SAUSAGE...,review,mRQzFZMGurB-3bP4CGTNpQ,"{u'funny': 3, u'useful': 1, u'cool': 0}"
2,qHmamQPCAKkia9X0uryA8g,2011-02-10,0_or2I5IGVERFYgL5j6-Iw,5,i don't care what other people say... top dog ...,review,2Rx-PAEZhkblWy4LNW_DEw,"{u'funny': 0, u'useful': 0, u'cool': 0}"


In [26]:
# find earliest and latest review date for each restaurant
mindate = reviews_final['date'].groupby([reviews_final['business_id']]).min()
maxdate = reviews_final['date'].groupby([reviews_final['business_id']]).max()
date_min_and_max = pd.concat([mindate, maxdate], 
                               axis=1, keys=['min', 'max'])
date_min_and_max.head(3)

,min,max
business_id,,
-0iqnv7MjKrgh7Q7bYRlUQ,2011-01-29,2012-10-09
-5Rah4ZvWsDu4oilUZxhtw,2006-05-03,2012-09-23
-6ncX3fnQ9OLCjfiUe-kFg,2006-01-11,2012-10-01


In [27]:
# check that reviews are at least a month long
date_min_and_max['difference'] = date_min_and_max['max'] - date_min_and_max['min']
date_min_and_max.sort('difference').head(3)

,min,max,difference
business_id,,,
0gE8y94ytFFdFDDNon-K0g,2012-02-18,2012-10-11,236 days
R2-_hZIboGriSatS2kn7rw,2012-01-27,2012-10-15,262 days
huHxwjbkyEf1nTam-o2BOQ,2012-01-12,2012-10-15,277 days


In [28]:
# write final dfs as csv files
business_final.to_csv('business_final_academic.csv', encoding='utf-8')
reviews_final.to_csv('review_final_academic.csv', encoding='utf-8')

In [29]:
reviewtest = pd.read_csv('review_final_academic.csv')
reviewtest.head(3)

,Unnamed: 0,business_id,date,review_id,stars,text,type,user_id,votes
0,0,qHmamQPCAKkia9X0uryA8g,2006-09-23,M8G9Rs21i4euIo3T5gyGOg,4,Are you drunk? Is it around 3am? Are you in do...,review,Xsp1amevfceAqAMjKhZkgA,"{u'funny': 0, u'useful': 1, u'cool': 0}"
1,1,qHmamQPCAKkia9X0uryA8g,2011-04-06,o9HYfNDSACBPRykq8t21PQ,3,OH NO HE DIDN'T.\n\nTHE GUY MAKING THE SAUSAGE...,review,mRQzFZMGurB-3bP4CGTNpQ,"{u'funny': 3, u'useful': 1, u'cool': 0}"
2,2,qHmamQPCAKkia9X0uryA8g,2011-02-10,0_or2I5IGVERFYgL5j6-Iw,5,i don't care what other people say... top dog ...,review,2Rx-PAEZhkblWy4LNW_DEw,"{u'funny': 0, u'useful': 0, u'cool': 0}"


In [30]:
businesstest = pd.read_csv('business_final_academic.csv')
businesstest.head(3)

,Unnamed: 0,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,photo_url,review_count,schools,stars,state,type,url
0,0,qHmamQPCAKkia9X0uryA8g,"[Restaurants, Hot Dogs]",Berkeley,"2534 Durant Ave\nTelegraph Ave\nBerkeley, CA 9...",37.867894,-122.257433,Top Dog,"[Telegraph Ave, UC Campus Area]",1,http://s3-media4.ak.yelpcdn.com/bphoto/nFfca2I...,1270,[University of California at Berkeley],4.5,CA,business,http://www.yelp.com/biz/top-dog-berkeley
1,1,JrGSfjRqAtIZjjwC3FFQ4w,"[American (Traditional), Barbeque, Restaurants]",San Luis Obispo,"1001 Higuera St\nSte A\nSan Luis Obispo, CA 93401",35.281110,-120.660760,Firestone Grill,[],1,http://s3-media4.ak.yelpcdn.com/bphoto/p3Gp2uN...,1153,[California Polytechnic State University],4.5,CA,business,http://www.yelp.com/biz/firestone-grill-san-lu...
2,2,VFslQjSgrw4Mu5_Q1xk1KQ,"[Steakhouses, Brazilian, Restaurants]",Palo Alto,"529 Alma St\nPalo Alto, CA 94301",37.442975,-122.163048,Pampas,[],1,http://s3-media4.ak.yelpcdn.com/bphoto/Wn0OeAX...,1146,[Stanford University],4.0,CA,business,http://www.yelp.com/biz/pampas-palo-alto
